# Running antismash in chunks

In [1]:
import os
import pandas as pd
import numpy as np

workDir = '/home/sam/FullCyc_metagenome/annotation/antismash'
enr_gene_gff = os.path.join(workDir, 'blocked_data', 'enr_genes_n1.gff')
contig_fasta = os.path.join(workDir, 'IMG_contigs.fasta')

chunks = 100
cpus = 20

## Split up gff and contigs

In [2]:
enr_gene_gff_gf = pd.read_csv(enr_gene_gff, sep='\t', header = None)

In [3]:
enr_gene_gff_gf_split = np.array_split(enr_gene_gff_gf, chunks)

### Split up the GFF3 file

In [4]:
for i in range(0, len(enr_gene_gff_gf_split)):
    enr_gene_gff_gf_split[i].to_csv(os.path.join(workDir, 'blocked_data', 'tmp_blocked_data.gff'), 
                                    header=False, index=False, sep='\t')
    cmd = 'cat ' + os.path.join(workDir, 'blocked_data', 'gffheader.txt') + ' > ' + os.path.join(workDir, 'blocked_data', 'chunk_' + str(i) + '.gff')
    os.system(cmd)
    cmd = 'cat ' + os.path.join(workDir, 'blocked_data', 'tmp_blocked_data.gff') + ' >> ' + os.path.join(workDir, 'blocked_data', 'chunk_' + str(i) + '.gff')
    os.system(cmd)

### Split up the contig fasta file

In [5]:
for i in range(0, len(enr_gene_gff_gf_split)):
    contig_list = list(set(enr_gene_gff_gf_split[i][0]))
    with open(os.path.join(workDir, 'blocked_data', 'temp_contig_list.txt'), 'w') as temp_file:
        for contig in contig_list:
            temp_file.write('>' + contig + '$\n')
    cmd = 'grep -A 1 -f ' + os.path.join(workDir, 'blocked_data', 'temp_contig_list.txt') + ' ' + contig_fasta + ' > ' + os.path.join(workDir, 'blocked_data', 'chunk_' + str(i) + '.fasta')
    os.system(cmd)
    

## Run Antismash for each chunk

In [ ]:
for i in range(0, len(enr_gene_gff_gf_split)):
    print('Running chunk ' + str(i) + '\n')
    outdir = os.path.join(workDir, 'blocked_data', 'chunk_' + str(i) + '_output')
    if not os.path.exists(outdir):
        cmd = 'mkdir ' + outdir
        os.system(cmd)
        cmd = ' '. join(['antismash --taxon bacteria', 
                         '--cpus', str(cpus), 
                         '--genefinding-gff3', os.path.join(workDir, 'blocked_data', 'chunk_' + str(i) + '.gff'), 
                         '--output-dir', outdir,
                         os.path.join(workDir, 'blocked_data', 'chunk_' + str(i) + '.fasta')])
        os.system(cmd)
    else:
        print(outdir + '   EXISTS!!!!!')
    print('\n---\n')